# In this step, we'll assign money/award codes based on prior award letter analysis

_Definition for award codes shown in the "Out of Pocket Cost" column: average unmet need for 0 EFC students BEFORE loans_									
+++: Less than \\$5,000  (i.e. no family contribution after Stafford loans)   
++: Between \\$5,000-\\$8,000 (i.e. no more than \\$2,500 in need after Stafford)   
++/-: Most awards ++, but some not as good   
+/--: Most awards are bad, but some are good   	
+/---: Almost all awards are bad, but we had a few surprises   		
--: \\$12,000-\\$15,000   
---: >\\$15,000

_Note that the year references below are from summer 2020; they should be updated each year_
### Here are the fields we're working with. Most were calculated in a separate workbook using student level data
- Noble awards collected FY20 (most recent year)
- Noble mean unmet need FY20 (most recent year)
- Noble min unmet need FY20 (most recent year)
- Noble max unmet need FY20 (most recent year)
- Noble money range FY21 __(we will be calculating this value below)__
- Noble money range FY20 (import from last year--based on FY19 awards)
- Noble money range FY19 (import from prior year--based on FY18 awards)
- Noble money code FY21 (follow rules--see below)
- Noble money code FY20 (import from last year--based on FY19 awards)
- Noble Money FY21 (1 if two plus signs, 0 otherwise based on Money code FY21)
- Current Noble students (pulled from Salesforce: attending plus matriculating to)

### We're also going to grab the NetPrice0-30 number along with Public/Private and State designation from the sfa1718 and  BaseDir; these will help us infer values where we don't have actual awards (net price is given is given in such a way to let us infer public vs. private)


In [1]:
import pandas as pd
import numpy as np
import os

# Edit these to reflect any changes
work_location = 'inputs'
base_dir = 'base_dir.csv'
fin_aid = 'sfa1718.csv'  # INCREMENT
noble_awards = '../../raw_inputs/financial_aid_analysis_output.xlsx'  # UPDATE THIS FILE EACH YEAR!
fin_aid_output = 'award_info_final.xlsx'
new_range = "Noble money range FY21"  # INCREMENT
new_code = "Noble money code FY21"  # INCREMENT
old_range = "Noble money range FY20"  # INCREMENT
old_code = "Noble money code FY20"  # INCREMENT
new_money = "Noble Money FY21"  # INCREMENT
old_money = "Noble Money FY20"  # INCREMENT

In [2]:
os.chdir(work_location)

In [4]:
# First, we'll load the source files and combine them
df = pd.concat([pd.read_csv(base_dir, usecols=["UNITID", "INSTNM", "STABBR"], index_col="UNITID"),
                pd.read_excel(noble_awards, index_col="UNITID"),
               ], axis=1)
# Now, we don't want any net price colleges if they weren't already in base_dir or our award letter info
# so we'll do a left join. Note that either the Public or Private column is populated, never both
a_df = pd.read_csv(fin_aid, index_col="UNITID",
                   usecols=["UNITID", "NPT412", "NPIS412"]).rename(
                        columns={"NPT412":"NetPricePrivate0-30", "NPIS412":"NetPricePublic0-30"})
a_df = a_df[~pd.isnull(a_df["NetPricePublic0-30"]) | ~pd.isnull(a_df["NetPricePrivate0-30"])]
df=pd.merge(df, a_df, how="left", left_index=True, right_index=True)
df.head()

c:\users\mniksch\dropbox (nnocs)\documents\noblegit\2020\annual-counseling\.env\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


,INSTNM,STABBR,Noble awards collected FY20,Noble mean unmet need FY20,Noble median unmet need FY20,Noble min unmet need FY20,Noble max unmet need FY20,Noble money code FY20,Noble money range FY20,Noble money code FY19,Noble money range FY19,Current Noble Students,Name,NetPricePublic0-30,NetPricePrivate0-30
UNITID,,,,,,,,,,,,,,,
100654,Alabama A & M University,AL,30.0,17492.0,19799.0,5029.0,27311.0,++/-,">$15k, some <$6k",++/-,"$12k-$15k, some <$6k",32.0,Alabama A & M University,13893.0,NaN
100663,University of Alabama at Birmingham,AL,NaN,NaN,NaN,NaN,NaN,---,>$15k,NaN,NaN,1.0,University of Alabama at Birmingham,14550.0,NaN
100690,Amridge University,AL,NaN,NaN,NaN,NaN,NaN,++/-,$8k-$10k,--,$10k-$12k,NaN,NaN,NaN,15322.0
100706,University of Alabama in Huntsville,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17561.0,NaN
100724,Alabama State University,AL,1.0,24747.0,24747.0,24747.0,24747.0,NaN,NaN,---,>$15k,7.0,Alabama State University,11344.0,NaN


### Use the the numbers we have to set the "money range FY21" using the following rules:
- If there are at least 3 award letters, use the mean to set the ranges into these discrete categories:
    - <\\$0   
    - <\\$5k   
    - \\$5k-\\$8k   

### Then, repeat those three categories for any Illinois schools with NetPrice0-30 in those ranges or any national private schools with the same ranges. These are all automatic "money" schools, so we don't have to worry about the outliers as much

In [5]:
# The code below will take a first pass at assigning values
passed_fields = [
    "Noble awards collected FY20",  # INCREMENT
    "Noble mean unmet need FY20",  # INCREMENT
    "Noble max unmet need FY20",  # INCREMENT
    "Noble min unmet need FY20",  # INCREMENT
    "STABBR",
    "NetPricePublic0-30",
    "NetPricePrivate0-30",
]
def range_checker(mean_unmet, min_unmet):
    """returns a number range, min could be None to handle net price data"""
    if mean_unmet < 0:
        return "<$0"
    elif mean_unmet <= 5000:
        return "<$5k"
    elif mean_unmet <= 8000:
        return "$5k-$8k"
    elif mean_unmet <= 10000:
        if min_unmet is not None and min_unmet <= 5000:
            return "$8k-$10k, some <$5k"
        else:
            return "$8k-$10k"
    elif mean_unmet <= 12000:
        if min_unmet is not None and min_unmet <= 6000:
            return "$10k-$12k, some <$6k"
        elif min_unmet is not None and min_unmet <= 8000:
            return "$10k-$12k, some <$8k"
        else:
            return "$10k-$12k"
    elif mean_unmet <= 15000:
        if min_unmet is not None and min_unmet <= 6000:
            return "$12k-$15k, some <$6k"
        else:
            return "$12k-$15k"
    elif mean_unmet > 15000:
        if min_unmet is not None and min_unmet <= 6000:
            return ">$15k, some <$6k"
        elif min_unmet is not None and min_unmet <= 12000:
            return ">$15k, some <$12k"
        else:
            return "$>$15k"
    else:
        return "TBD"

def _create_award_ranges(x):
    num_awards, mean_unmet, max_unmet, min_unmet, state, public_np, private_np = x
    if num_awards >= 3:
        return range_checker(mean_unmet, min_unmet)
    else:
        if (not np.isnan(public_np)) and (state == "IL"):
            return range_checker(public_np, None)
        elif not np.isnan(private_np):
            return range_checker(private_np, None)
        else:
            if num_awards > 0:
                if mean_unmet > 15000:
                    return ">$15k"
                else:
                    return "CHECK BY HAND"
            return "?"
df[new_range] = df[passed_fields].apply(_create_award_ranges, axis=1)

In [6]:
# Now that we have ranges, assign money codes and money yes/no
def money_code(money_range):
    """Returns the money code for a given range"""
    translation = {
        '$10k-$12k': '--',
        '$10k-$12k, some <$6k': '++/-',
        '$10k-$12k, some <$8k': '++/-',
        '$12k-$15k': '--',
        '$12k-$15k, some <$6k': '++/-',
        '$5k-$8k': '++',
        '$8k-$10k': '++/-',
        '$8k-$10k, some <$5k': '++/-',
        '?': '?',
        '<$0': '+++',
        '<$5k': '+++',
        '>$15k': '---',
        '>$15k, some <$12k': '+/---',
        '>$15k, some <$6k': '+/--',
        '>$15k, some <$6k': '++/-',
    }
    if money_range in translation:
        return translation[money_range]
    else:
        return '?'
def is_money(money_code):
    """Yields a zero or one for a binary impression of "Money" """
    if money_code in ['+++', '++', '++/-']:
        return 1
    else:
        return 0

In [7]:
df[new_code] = df[new_range].apply(money_code)
df[new_money] = df[new_code].apply(is_money)
df[old_money] = df[old_code].apply(is_money)

In [8]:
# We'll need to review the results of the above manually, this helps us target rows
def _check_needed(x):
    """Gives a single field flagging a row for review"""
    this_code, last_code, this_money, last_money, this_range = x
    if this_range == "CHECK BY HAND":
        return "CHECK BY HAND (small # of awards colleced)"
    elif this_money > last_money:
        return "CHECK NEW MONEY"
    elif this_money < last_money:
        return "CHECK LOST MONEY"
    elif (this_money == 1) and ("some" in this_range):
        return "CHECK SIZE OF SOME"
    else:
        return "NO CHECK"
df["Check"] = df[[new_code, old_code, new_money, old_money, new_range]].apply(_check_needed, axis=1)

In [9]:
# Save the current results for the manual step described below
df.to_excel('temporary_fin_aid_output.xlsx')

## Now open the temporary Excel file just saved and filter on the "Check" column for each reason:

For each of these cases, you're only going to edit the "money range" for this most recent year; money code and yes/no will be updated in the code.

### CHECK LOST MONEY: Drop from Money from prior year
1. Filter first to all schools that have 0 awards collected. These are the most suspicious because we're only relying on federal net price info.
2. Filter on "Noble money range FY21"="?". For all of these schools, assign the value instead to what's in "Noble money range FY20" and change the "Check" column to "NO CHECK".
3. Next, remove the filter on "Noble money range FY21" and filter "Current Noble Student"!=0. For all of these schools, also copy the FY20 money range and change to "NO CHECK". (We're trusting the fact that Noble students have afforded the schools in the past over a "Net Price" number that's high and two years old.)
4. For the remaining schools with no awards collected, accept the new money range and change to "NO CHECK". Remove the filter on "Noble awards collected FY20" and instead look at schools with at least 5 awards collected.
5. For all schools with "Noble min unmet need FY20">10,000, accept the result and change to "NO CHECK". These appear to be real cases of the college becoming less generous and we should send fewer applications next year.
6. For the remaining schools with at least 5 awards collected, look at the "Noble median unmet need FY20" column. Apply the rules in the "range_checker" formula above to the median unmet need. (We originally used the mean.) 
7. Unfilter the "Noble awards collected FY20" column. The only "CHECK LOST MONEY" schools left should be ones with a small number of awards collected. We will simply repeat last year's money range for all of these. Change the values and move to the next section.

### CHECK NEW MONEY: Increase in Money from prior year
1. Filter first to all schools that have 0 awards collected. We don't trust these schools as they'll typically be a bunch of small schools that might have errors. For schools with ranges last year, assign this year to those values. For schools with no ranges last year, use "$10k-$12k".
2. Now we have the schools that jumped because of awards. For schools with a set range (e.g. "$8k-$10k"), accept the change by indicating "NO CHECK". (THis is still a speculatively money school, but decent enough chance that students should give it a shot.)
3. For schools with a value like "$10k-$12k, some <$6k", look at the raw awards for each school. (You will have these from a separate process.) If it was only a single award letter that was causing the exception, change the range to the range without the exception. After you've checked each of these schools, "CHECK NEW MONEY" should be complete.

### CHECK BY HAND: small # of awards collected
1. First filter by schools with a Noble money range in the prior year. You will likely copy the range from last year, but look at the award ranges to see if any look very off. As an example, if the school was not a money school last year, but appears to have two awards that were money this most recent year, then amend the value to a range based on the two awards.
2. For schools with no money range last year there are two options. If the 1-2 awards are not money, enter a range based on the actual award(s). If the 1-2 awards do indicate a money school, we won't trust it--instead enter "$10k-$12k"

### CHECK SIZE OF SOME: "some" entries that are in the money--one or two outliers might be sneaking it into money
1. For all of the entries with median awards <10k, go ahead and accept the result--under 10k is money anyway. (We use mean above, but the purpose of this check is to see if there was only 1-2 good awards. A "money" median protects from that result.)
2. For the remaining schools, if the mean and median are <20k and the school was money last year, also accept the result.
3. For any remaining schools, (with high means or medians), only accept the result if at least 20% of the awards are money. Otherwise change the range to remove the ", some <$6k" exception.
4. With the last line, you should have fully processed all of the "CHECK"s.


### Once this is done, save the file as "edited_fin_aid_output.xlsx"
---
### We'll then load the file back and save what we need for the final directory

In [10]:
df = pd.read_excel('edited_fin_aid_output.xlsx', index_col="UNITID")

In [11]:
df.head(3)

,INSTNM,STABBR,Noble awards collected FY20,Noble mean unmet need FY20,Noble median unmet need FY20,Noble min unmet need FY20,Noble max unmet need FY20,Noble money code FY20,Noble money range FY20,Noble money code FY19,Noble money range FY19,Current Noble Students,Name,NetPricePublic0-30,NetPricePrivate0-30,Noble money range FY21,Noble money code FY21,Noble Money FY21,Noble Money FY20,Check
UNITID,,,,,,,,,,,,,,,,,,,,
100654,Alabama A & M University,AL,30.0,17492.0,19799.0,5029.0,27311.0,++/-,">$15k, some <$6k",++/-,"$12k-$15k, some <$6k",32.0,Alabama A & M University,13893.0,NaN,">$15k, some <$6k",++/-,1,1,NO CHECK
100663,University of Alabama at Birmingham,AL,NaN,NaN,NaN,NaN,NaN,---,>$15k,NaN,NaN,1.0,University of Alabama at Birmingham,14550.0,NaN,?,?,0,0,NO CHECK
100690,Amridge University,AL,NaN,NaN,NaN,NaN,NaN,++/-,$8k-$10k,--,$10k-$12k,NaN,NaN,NaN,15322.0,$>$15k,?,0,1,NO CHECK


In [12]:
# Fix the money code after any potential corrections:
df[new_code] = df[new_range].apply(money_code)
df[new_money] = df[new_code].apply(is_money)

In [14]:
df.drop(columns=[
    "STABBR",
    "NetPricePublic0-30",
    "NetPricePrivate0-30",
    "Noble money range FY19",  # INCREMENT
    "Noble money code FY19",  # INCREMENT
    "Noble Money FY20",  # INCREMENT
    "Check",
], inplace=True)
df["Current Noble Students"] = df["Current Noble Students"].apply(lambda x: 0 if pd.isnull(x) else x)

In [15]:
df.head(3)

,INSTNM,Noble awards collected FY20,Noble mean unmet need FY20,Noble median unmet need FY20,Noble min unmet need FY20,Noble max unmet need FY20,Noble money code FY20,Noble money range FY20,Current Noble Students,Name,Noble money range FY21,Noble money code FY21,Noble Money FY21
UNITID,,,,,,,,,,,,,
100654,Alabama A & M University,30.0,17492.0,19799.0,5029.0,27311.0,++/-,">$15k, some <$6k",32.0,Alabama A & M University,">$15k, some <$6k",++/-,1
100663,University of Alabama at Birmingham,NaN,NaN,NaN,NaN,NaN,---,>$15k,1.0,University of Alabama at Birmingham,?,?,0
100690,Amridge University,NaN,NaN,NaN,NaN,NaN,++/-,$8k-$10k,0.0,NaN,$>$15k,?,0


In [16]:
# Now save this off. Eventually, we'll add this to the combine step in #4, but right now, we're adding it to
# the output of step #4 using a vlookup
df.to_excel(fin_aid_output, index_label="UNITID", na_rep="N/A")